In [ ]:
from pathlib import Path
from freqtrade.configuration import Configuration

import pandas as pd
import numpy as np
import statistics
import math
# Customize these according to your needs.s

# Initialize empty configuration object
path='\\freqtrade\\user_data\\config\\'

config = Configuration.from_files([path + "config_binance_zone.json"])

# Optionally, use existing configuration file
# config = Configuration.from_files(["config.json"])

# Define some constants
config["timeframe"] = "4h"
# Name of the strategy class
config["strategy"] = "YoyoActionStrategy"
# Location of the data
data_location = Path(config['user_data_dir'], 'data', 'binance')
# Pair to analyze - Only use osne pair here
pair = "BTC_USDT"

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2021-05-26_11-32-09.json"

In [ ]:
from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair)

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
candles.head()

In [ ]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy(config)

# Generate buy/sell signals using strategy 
df = strategy.analyze_ticker(candles, {'pair': pair})
df.tail()

In [ ]:
print(f"Generated {df['buy'].sum()} buy signals")
data = df.set_index('date', drop=False)
data.tail()

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats


In [ ]:
backtest_dir

In [ ]:
stats = load_backtest_stats(backtest_dir)

strategy = config["strategy"]
# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])

# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])

In [ ]:
strategy_comparison = stats['strategy_comparison']
stats['strategy']['YoyoActionStrategy'].keys()

In [ ]:
stats['strategy']['YoyoActionStrategy']['trades']

In [ ]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair)

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
candles.head()

In [ ]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy(config)

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {'pair': pair})
df.tail()

In [ ]:
# Report results
print(f"Generated {df['buy'].sum()} buy signals")
data = df.set_index('date', drop=False)
data.tail()

In [ ]:
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

In [ ]:
trades

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '4h')

parallel_trades.plot()

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]
data.set_index('date', drop=False)
data_red = data['2017-01-01':'2021-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['ema_fast', 'ema_slow', 'trail2','greenLine'],
                                   indicators2=['rsi']
                                  )

graph.show(renderer="browser")


In [ ]:
 trades.loc[trades['pair'] == pair]

In [ ]:
trades.columns


In [ ]:
trades.loc[trades['pair'] == pair,['pair','stake_amount','profit_ratio','profit_abs','stop_loss_abs','stop_loss_ratio','min_rate','max_rate']]

In [ ]:
trades['maximum_lost'] = trades.groupby("pair")['profit_ratio'].agg('min')
# trades.groupby("pair")["sell_reason"].value_counts()
trades.loc[:,['pair','stake_amount','profit_ratio','profit_abs','stop_loss_abs','stop_loss_ratio','min_rate','max_rate','maximum_lost']]

In [ ]:

summaryTrade = trades.groupby("pair").agg(
    maximum_lost = pd.NamedAgg(column='profit_ratio' , aggfunc=min),
    #trades_count = pd.NamedAgg(column='profit_ratio', aggfunc=count),
    trades_count = pd.NamedAgg(column='profit_ratio', aggfunc=lambda x:x.count())
    
)
summaryTrade['win_count'] = trades.loc[(trades['profit_ratio'] > 0 )].groupby("pair")['profit_ratio'].count()
summaryTrade['loss_count'] = trades.loc[(trades['profit_ratio'] < 0 )].groupby("pair")['profit_ratio'].count()
summaryTrade['win_ration'] = trades.loc[(trades['profit_ratio'] > 0 )].groupby("pair")['profit_ratio'].count()/summaryTrade['trades_count']
summaryTrade['loss_ration'] = trades.loc[(trades['profit_ratio'] < 0 )].groupby("pair")['profit_ratio'].count()/summaryTrade['trades_count']

summaryTrade['win_avg'] = trades.loc[(trades['profit_ratio'] > 0 )].groupby("pair")['profit_ratio'].mean()
summaryTrade['loss_avg'] = trades.loc[(trades['profit_ratio'] < 0 )].groupby("pair")['profit_ratio'].mean()
summaryTrade['riskreward'] = abs(summaryTrade['win_avg']/summaryTrade['loss_avg']) 

summaryTrade['std_win'] =  trades.loc[(trades['profit_ratio'] > 0 )].groupby("pair")['profit_ratio'].apply(lambda x: statistics.stdev(x))
summaryTrade['std_loss'] = trades.loc[(trades['profit_ratio'] < 0 )].groupby("pair")['profit_ratio'].apply(lambda x: statistics.stdev(x))
summaryTrade['std_all'] =  trades.groupby("pair")['profit_ratio'].apply(lambda x: statistics.stdev(x))
summaryTrade['std_riskreward'] = summaryTrade['std_win']/summaryTrade['std_loss']

summaryTrade

In [ ]:
statistics.stdev(trades.loc[(trades['profit_ratio'] < 0 ),'profit_ratio'])